Computation of the Shannon index for copy numbers of different cell types (including code for Fig. 4F)

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import random
import glob
from operator import itemgetter
import itertools
from itertools import groupby
import seaborn as sns
import matplotlib.ticker as ticker
import sys
import scipy.stats  as stats
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
import statsmodels.api as sm

In [ ]:
tumor_roi_dat = pd.read_csv('new_annotation_Feb2021_KW.txt', delimiter = "\t")
protein_dat=pd.read_csv('ERCC_IgG_norm.csv')
nuc_count = pd.read_csv('nuc_count.csv')

protein_dat=protein_dat.rename({'Unnamed: 0': 'Protein'}, axis=1) 

all_rois=list(protein_dat.columns)[1:]

tumor_core_roi_dict={}
excluded_rois=[]
for loc in range(len(tumor_roi_dat)):
    cur_row = tumor_roi_dat.iloc[loc]
    roi = cur_row['ROI']
    roi_num=roi.split('_')[1]
    if len(roi_num)==1:
        roi = 'ROI_0'+roi_num
    #print(cur_row)
    if roi in all_rois:
        key = (cur_row['Tumor_ID'],cur_row['Core_ID'])
        if key[0]==21 or key[0]==8:
            excluded_rois.append(roi)
        else:
            if key in tumor_core_roi_dict:
                tumor_core_roi_dict[key].append(roi)
            else:
                tumor_core_roi_dict[key] = [roi]            
            
#protein_dat=protein_dat.drop(excluded_rois,1)

tumor_ids=list(tumor_roi_dat['Tumor_ID'])
core_ids=list(tumor_roi_dat['Core_ID'])
fish_cores=list(tumor_roi_dat['FISH_core'])

tumor_core_to_shreya_base={(tumor_ids[i],core_ids[i]):fish_cores[i] for i in range(len(tumor_roi_dat))}
tumor_core_to_shreya={}
for k,v in tumor_core_to_shreya_base.items():
    if k in tumor_core_roi_dict.keys():
        tumor_core_to_shreya[k]=v
all_proteins_cores_shreya=list(tumor_core_to_shreya.values())

tumor_roi_dict={}
for k, rois in tumor_core_roi_dict.items():
    tumor=k[0]
    if tumor in tumor_roi_dict.keys():
        tumor_roi_dict[tumor]=tumor_roi_dict[tumor]+rois
    else:
        tumor_roi_dict[tumor]=rois
            
nuc_count=nuc_count.rename({'Unnamed: 0': 'ROI'}, axis=1) 

tumor_protein_dfs=[]
for tumor,rois in tumor_roi_dict.items():
    cur_df=protein_dat[rois]
    cur_df.index=protein_dat['Protein']
    tot_nuc_count=nuc_count[nuc_count['ROI'].isin(protein_dat[rois])]['nuclei_count'].sum()
    for roi in rois:
        cur_weight=list(nuc_count[nuc_count['ROI']==roi]['nuclei_count'])[0]/tot_nuc_count
        cur_df[roi]=cur_df[roi]*cur_weight 
    cur_df=cur_df.sum(axis=1).to_frame().rename(columns={0:tumor})
    tumor_protein_dfs.append(cur_df)
protein_dat_by_tumor=pd.concat(tumor_protein_dfs,1)

protein_dat_by_tumor=protein_dat_by_tumor.T

In [ ]:
filenames = glob.glob("FISH_data/table*")
dfs_o = {}

for filename in filenames:
    df_o = pd.read_csv(filename)
    shreya_core_num = int(filename.split('_')[2])
    image_num = int(filename.split('_')[3].split('.')[0])
    dfs_o[(shreya_core_num,image_num)]=df_o

    
HI=6
MAX_HI = 50

def geno_hi(geno,df):
    return (df[geno]>=HI)&(df[geno]<=MAX_HI)
def geno_lo(geno,df):
    return (df[geno]>=1)&(df[geno]<HI)

for df_fish in list(dfs_o.values()):
    df_fish['EGFR_amp']=1*(geno_hi('EGFR',df_fish))
    df_fish['PDGFRA_amp']=1*(geno_hi('PDGFRA',df_fish))
    df_fish['CDK4_amp']=1*(geno_hi('CDK4',df_fish))
    
dfs={}    
for key,df in dfs_o.items():

    if key[0] not in tumor_core_to_shreya.values():
        continue

    df['EGFR_CDK4_amp']=1*((df['EGFR']>=HI)&(df['CDK4']>=HI))
    df['EGFR_NOT_CDK4_amp']=1*((df['EGFR']>=HI)&(df['CDK4']<HI))
    df['CDK4_NOT_EGFR_amp']=1*((df['CDK4']>=HI)&(df['EGFR']<HI))    
    df['Non_amp']=1*((df['EGFR']<HI)&(df['CDK4']<HI))
    
    dfs[key]=df[(df['EGFR']<=MAX_HI)&(df['PDGFRA']<=MAX_HI)&(df['CDK4']<=MAX_HI)]
  

In [ ]:
shreya_to_tumor={v:k[0] for k,v in tumor_core_to_shreya.items()}
core_list=list(set([item[0] for item in dfs.keys()]))
tumor_list = list(set([shreya_to_tumor[core] for core in core_list]))

tumor_image_dict={}
for k in dfs.keys():
    this_tumor=shreya_to_tumor[k[0]]
    if this_tumor in tumor_image_dict.keys():
        tumor_image_dict[this_tumor]=tumor_image_dict[this_tumor]+[k]
    else:
        tumor_image_dict[this_tumor]=[k]
        
dfs_in_tumor={}
for tumor, images in tumor_image_dict.items():
    dfs_in_tumor[tumor]=pd.concat([dfs[image] for image in images])

In [ ]:
tumor_OR_df=pd.read_csv('tumor_OR_df.csv').rename(columns={'Unnamed: 0':'Tumor'})
tumor_OR_df.index=tumor_OR_df['Tumor']
tumor_OR_df=tumor_OR_df.drop(['Tumor'],1)

In [ ]:
ORhigh_tumors=list(tumor_OR_df[tumor_OR_df['OR_char']=='OR, top 1/3'].index)
ORmiddle_tumors=list(tumor_OR_df[tumor_OR_df['OR_char']=='OR, middle 1/3'].index)
ORlow_tumors=list(tumor_OR_df[tumor_OR_df['OR_char']=='OR, bottom 1/3'].index)

In [ ]:
def shannon_in_group(tumor_group,geno_def,geno):
    tumor_stats=[]
    for tumor in tumor_group:
        tumor_df=dfs_in_tumor[tumor]
        all_copy_nums=tumor_df[tumor_df[geno_def]==1][geno]
        all_amp=len(tumor_df[tumor_df[geno_def]==1])
        probs=np.array(all_copy_nums.value_counts()/all_amp) 
        shannon_ind=-sum([prob*np.log(prob) for prob in probs])
        tumor_stats.append(shannon_ind)
    return tumor_stats

In [ ]:
tumor_groups={'Top OR tertile':ORhigh_tumors,'Middle OR tertile':ORmiddle_tumors,'Bottom OR tertile':ORlow_tumors}
shannon_dfs={}

group_dfs=[]
for group_name,tumor_group in tumor_groups.items():
    this_dict={}
    vals=shannon_in_group(tumor_group,'EGFR_NOT_CDK4_amp','EGFR')
    this_dict['Copy number Shannon index']=vals
    this_dict['Tertile']=[group_name for i in range(len(vals))]
    this_dict['Type']=['EGFR copy number Shannon, E cells' for i in range(len(vals))]
    dist_name='EGFR-only copy number'
    group_dfs.append(pd.DataFrame.from_dict(this_dict,orient='index').T)
shannon_dfs[dist_name]=pd.concat(group_dfs)

group_dfs=[]
for group_name,tumor_group in tumor_groups.items():
    this_dict={}
    vals=shannon_in_group(tumor_group,'EGFR_CDK4_amp','EGFR')
    this_dict['Copy number Shannon index']=vals
    this_dict['Tertile']=[group_name for i in range(len(vals))]
    this_dict['Type']=['EGFR copy number Shannon, EC cells' for i in range(len(vals))]
    dist_name='EGFR dual-amp copy number'
    group_dfs.append(pd.DataFrame.from_dict(this_dict,orient='index').T)
shannon_dfs[dist_name]=pd.concat(group_dfs)

group_dfs=[]
for group_name,tumor_group in tumor_groups.items():
    this_dict={}
    vals=shannon_in_group(tumor_group,'CDK4_NOT_EGFR_amp','CDK4')
    this_dict['Copy number Shannon index']=vals
    this_dict['Tertile']=[group_name for i in range(len(vals))]
    this_dict['Type']=['CDK4 copy number Shannon, C cells' for i in range(len(vals))]
    dist_name='CDK4-only copy number'
    group_dfs.append(pd.DataFrame.from_dict(this_dict,orient='index').T)
shannon_dfs[dist_name]=pd.concat(group_dfs)

group_dfs=[]
for group_name,tumor_group in tumor_groups.items():
    this_dict={}
    vals=shannon_in_group(tumor_group,'EGFR_CDK4_amp','CDK4')
    this_dict['Copy number Shannon index']=vals
    this_dict['Tertile']=[group_name for i in range(len(vals))]
    this_dict['Type']=['CDK4 copy number Shannon, EC cells' for i in range(len(vals))]
    dist_name='CDK4 dual-amp copy number'
    group_dfs.append(pd.DataFrame.from_dict(this_dict,orient='index').T)
shannon_dfs[dist_name]=pd.concat(group_dfs)

shannon_full_df=pd.concat(list(shannon_dfs.values()))


In [ ]:
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = "22"
fig = plt.figure(figsize=(10,6))
ax=sns.boxplot(x="Type", y="Copy number Shannon index",hue="Tertile", palette=["salmon", 'lightgray',"lightsteelblue"], data=shannon_full_df,width=0.6)
plt.xlabel('')
plt.ylabel('Copy number\nShannon index')
#ax.legend_.remove()
plt.xticks(rotation=90)
#plt.savefig('shannon_boxplots.pdf')
plt.show()

In [ ]:
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = "28"
fig = plt.figure(figsize=(12,6))
ax=sns.boxplot(x="Type", y="Copy number Shannon index",hue="Tertile", palette=["salmon", 'lightgray',"lightsteelblue"], data=shannon_full_df,width=0.6)
plt.xlabel('')
plt.ylabel('Copy number\nShannon index')
ax.legend_.remove()
ax.set(xticklabels=[])
ax.set(xlabel=None)
plt.setp(ax.artists, edgecolor = 'black')
plt.setp(ax.lines, color='black')
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(1.2)
plt.tight_layout()
#plt.savefig('shannon_boxplots.pdf')
plt.show()

In [ ]:
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = "28"
fig = plt.figure(figsize=(12,6))
ax=sns.boxplot(x="Type", y="Copy number Shannon index",hue="Tertile", palette=["salmon", 'lightgray',"lightsteelblue"], data=shannon_full_df,width=0.6)
plt.xlabel('')
plt.ylabel('Copy number\nShannon index')
#ax.legend_.remove()
ax.set(xticklabels=[])
ax.set(xlabel=None)
plt.setp(ax.artists, edgecolor = 'black')
plt.setp(ax.lines, color='black')
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(1.2)
plt.tight_layout()
#plt.savefig('shannon_boxplots.pdf')
plt.show()